In [1]:
import pandas as pd
from pathlib import Path
import re
from unidecode import unidecode

In [2]:
maiores_cidades = [
    "JOINVILLE",
    "FLORIANOPOLIS",
    "BLUMENAU",
    "SAO JOSE",
    "ITAJAI",
    "CHAPECO",
    "PALHOCA",
    "CRICIUMA",
    "JARAGUA DO SUL",
    "LAGES",
    "BRUSQUE",
    "BALNEARIO CAMBORIU",
    "TUBARAO",
    "CAMBORIU",
    "NAVEGANTES",
    "RIO DO SUL",
    "ITAPEMA",
    "ARMAZEM",
    "CONCORDIA",
    "CACADOR",
    "ITUPORANGA",
    "SAO MIGUEL DO OESTE",
    "RIO NEGRINHO",
    "SAO BONIFACIO",
    "BIGUACU",
    "SAO FRANCISCO DO SUL",
    "ICARA",
    "GASPAR",
    "MAFRA",
    "IRATI",
    "PALMITOS",
    "LAJEADO GRANDE",
    "PIRATUBA",
    "IBIRAMA",
    "ENTRE RIOS",
    "ORLEANS",
    "BALNEARIO PICARRAS",
    "PORTO UNIAO",
    "BRACO DO NORTE",
    "SAO LOURENCO DO OESTE",
    "PORTO BELO",
    "PONTE ALTA DO NORTE",
]

anos = [i for i in range(2021, 2023)]

df_excel = pd.read_excel("../data/pop_residente.xlsx")
dfs = [pd.read_csv(f"../data/dataset/censo_{i}.csv", sep=";") for i in anos]

In [3]:
df_excel.head()

,Município,2001,2002,2003,2004,2005,2006,2008,2009,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Abdon Batista,2718,2676,2630,2534,2481,2428,2805,2805,2644,2635,2656,2643,2630,2617,2604,2577,2563,2548
1,Abelardo Luz,16828,17143,17469,18154,18533,18909,16870,16899,17151,17200,17584,17651,17717,17782,17847,17847,17904,17960
2,Agrolândia,7889,7942,8004,8133,8204,8275,9496,9661,9440,9552,9957,10115,10272,10427,10581,10711,10864,11013
3,Agronômica,4314,4359,4406,4506,4561,4615,4866,4925,4954,4985,5172,5239,5306,5371,5437,5385,5448,5509
4,Água Doce,6825,6854,6859,6870,6876,6882,6954,6959,6971,6979,7110,7121,7132,7143,7154,7138,7145,7152


In [4]:
dfs[0].head()

,Município,2021
0,420005 ABDON BATISTA,2656
1,420010 ABELARDO LUZ,17607
2,420020 AGROLANDIA,11076
3,420030 AGRONOMICA,6032
4,420040 AGUA DOCE,6720


In [5]:
for i, df in enumerate(dfs):
    df["Município"] = df["Município"].apply(lambda s : re.sub(r'^\d+\s*', '', s))

In [6]:
df_excel['Município'] = df_excel['Município'].apply(lambda x: unidecode(x))
df_excel['Município'] = df_excel['Município'].str.upper()

In [7]:
dfs = dfs[0].merge(
    dfs[1],
    how='inner',
    on='Município'
)

In [8]:
df_final = df_excel.merge(
    dfs,
    how='inner',
    on='Município'
)

In [9]:
df_final.head()

,Município,2001,2002,2003,2004,2005,2006,2008,2009,2011,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ABDON BATISTA,2718,2676,2630,2534,2481,2428,2805,2805,2644,...,2656,2643,2630,2617,2604,2577,2563,2548,2656,2642
1,ABELARDO LUZ,16828,17143,17469,18154,18533,18909,16870,16899,17151,...,17584,17651,17717,17782,17847,17847,17904,17960,17607,17652
2,AGROLANDIA,7889,7942,8004,8133,8204,8275,9496,9661,9440,...,9957,10115,10272,10427,10581,10711,10864,11013,11076,11197
3,AGRONOMICA,4314,4359,4406,4506,4561,4615,4866,4925,4954,...,5172,5239,5306,5371,5437,5385,5448,5509,6032,6152
4,AGUA DOCE,6825,6854,6859,6870,6876,6882,6954,6959,6971,...,7110,7121,7132,7143,7154,7138,7145,7152,6720,6678


In [11]:
anos_ignorar = [2007, 2010]

anos_drop = [f"{i}" for i in range(2001, 2016) if i not in anos_ignorar]

df_final = df_final.drop(columns=anos_drop)

In [14]:
df_final.columns = df_final.columns.astype(str)

In [15]:
df_final.to_parquet("../data/residentes.parquet")